# <span style="color:brown">Data Cleaning and Preprocessing with Pandas and Real World Dataset</span>


In [32]:
import pandas as pd
file_path = r'E:\Data Analysis\Infection.xlsx'
df = pd.read_excel(file_path)
df.head()

,AGE,BIRTH_DATE,HOURS,ORGAN_CLASSIFICATION,REPORT_DATE,MCROA_SENSITIVE,ANTIBIOTIC,INTENSITY
0,1 Year(s),2022-10-03,5 DAYS,GRAM -VE RODS,2022-10-26 12:14:02,Resistant To :,Ampicillin,Heavy
1,1 Year(s),2022-02-03,48 Hours,GRAM -VE RODS,2023-05-06 11:30:43,Resistant To :,Ampicillin,Light
2,1 Year(s),2022-11-08,48 Hours,GRAM -VE RODS,2023-06-12 10:25:00,Resistant To :,Ampicillin,Heavy
3,1 Year(s),NaT,24 Hours,GRAM -VE RODS,2023-05-19 10:27:43,Resistant To :,Chloramphenicol,Heavy
4,1 Year(s),2021-12-22,24 Hours,GRAM -VE RODS,2022-10-03 09:55:12,Sensitive To :,Amikacin,Heavy


In [33]:
df.dtypes

AGE                             object
BIRTH_DATE              datetime64[ns]
HOURS                           object
ORGAN_CLASSIFICATION            object
REPORT_DATE             datetime64[ns]
MCROA_SENSITIVE                 object
ANTIBIOTIC                      object
INTENSITY                       object
dtype: object

#  Missing values

In [34]:
print(df.isnull().sum().sum())

3634


In [35]:
print(df.isnull().sum())

AGE                       38
BIRTH_DATE                17
HOURS                     22
ORGAN_CLASSIFICATION      44
REPORT_DATE               60
MCROA_SENSITIVE           39
ANTIBIOTIC                33
INTENSITY               3381
dtype: int64


### Drop rows with missing values in the 'AGE' column and update the original DataFrame

In [36]:
df = df.dropna(subset=['AGE'])
print("Check change in AGE:")
print(df.isnull().sum())

Check change in AGE:
AGE                        0
BIRTH_DATE                 5
HOURS                     11
ORGAN_CLASSIFICATION      44
REPORT_DATE               53
MCROA_SENSITIVE           19
ANTIBIOTIC                26
INTENSITY               3378
dtype: int64


### Drop columns with missing values in the 'ANTIBIOTIC' column and update the orignal DataFrame

In [37]:
df.dropna(subset=['ANTIBIOTIC'], inplace=True)
print("Check change in ANTIBIOTIC:")
print(df.isnull().sum())

Check change in ANTIBIOTIC:
AGE                        0
BIRTH_DATE                 5
HOURS                     10
ORGAN_CLASSIFICATION      36
REPORT_DATE               31
MCROA_SENSITIVE            5
ANTIBIOTIC                 0
INTENSITY               3375
dtype: int64


### Fill with a specific value in 'BIRTH_DATE' : Placeholder or the most frequent value.

In [38]:
#Placeholder
most_frequent_birth_date = df['BIRTH_DATE'].mode()[0]
df['BIRTH_DATE'] = df['BIRTH_DATE'].fillna(most_frequent_birth_date)
missing_values = df.isnull().sum()
print("Check change in Hours:")
print(missing_values)


Check change in Hours:
AGE                        0
BIRTH_DATE                 0
HOURS                     10
ORGAN_CLASSIFICATION      36
REPORT_DATE               31
MCROA_SENSITIVE            5
ANTIBIOTIC                 0
INTENSITY               3375
dtype: int64


In [39]:
#Frequen value
most_frequent_organ = df['ORGAN_CLASSIFICATION'].mode()[0]
df['ORGAN_CLASSIFICATION'] = df['ORGAN_CLASSIFICATION'].fillna(most_frequent_organ)
missing_values = df.isnull().sum()
print("Check change ORGAN_CLASSIFICATION :")
print(missing_values)


Check change ORGAN_CLASSIFICATION :
AGE                        0
BIRTH_DATE                 0
HOURS                     10
ORGAN_CLASSIFICATION       0
REPORT_DATE               31
MCROA_SENSITIVE            5
ANTIBIOTIC                 0
INTENSITY               3375
dtype: int64


### Forward and fill missing values in 'REPORT_DATE' and 'MCROA_SENSITIVE' columns

In [40]:
df['REPORT_DATE'] = df['REPORT_DATE'].ffill()
df['MCROA_SENSITIVE'] = df['MCROA_SENSITIVE'].bfill()
missing_values = df.isnull().sum()
print("Check missing values after fill:")
print(missing_values)

Check missing values after fill:
AGE                        0
BIRTH_DATE                 0
HOURS                     10
ORGAN_CLASSIFICATION       0
REPORT_DATE                0
MCROA_SENSITIVE            0
ANTIBIOTIC                 0
INTENSITY               3375
dtype: int64


### Interpolate missing values in the 'HOURS' column

In [41]:
df.dtypes

AGE                             object
BIRTH_DATE              datetime64[ns]
HOURS                           object
ORGAN_CLASSIFICATION            object
REPORT_DATE             datetime64[ns]
MCROA_SENSITIVE                 object
ANTIBIOTIC                      object
INTENSITY                       object
dtype: object

### parse hours and convert object data type into float for Interpolate missing values in HOURS

In [42]:
def parse_hours(hours_str):
    if isinstance(hours_str, str):
        if 'DAYS' in hours_str:
            return int(hours_str.split()[0]) * 24 
        elif 'Hours' in hours_str:
            return int(hours_str.split()[0])
        else:
            return None 
    else:
        return hours_str  


df['HOURS'] = df['HOURS'].apply(parse_hours)

print("See Changes in datatypes")
df.dtypes



See Changes in datatypes


AGE                             object
BIRTH_DATE              datetime64[ns]
HOURS                          float64
ORGAN_CLASSIFICATION            object
REPORT_DATE             datetime64[ns]
MCROA_SENSITIVE                 object
ANTIBIOTIC                      object
INTENSITY                       object
dtype: object

In [43]:
# Interpolate missing values in 'HOURS' column
df['HOURS'] = df['HOURS'].interpolate()

missing_values = df.isnull().sum()
print("Check Hours coulmn:")
print(missing_values)

Check Hours coulmn:
AGE                        0
BIRTH_DATE                 0
HOURS                      0
ORGAN_CLASSIFICATION       0
REPORT_DATE                0
MCROA_SENSITIVE            0
ANTIBIOTIC                 0
INTENSITY               3375
dtype: int64


###  Custom Mapping or Business Rules for 'INTENSITY' missing values

#### Function to convert age strings to numeric years #####

In [44]:
def convert_age(age_string):
    try:
        if 'Year' in age_string:
            age = int(age_string.split()[0]) 
        elif 'Month' in age_string:
            age = round(int(age_string.split()[0]) / 12, 2) 
        else:
            age = None  
        return age
    except:
        return None

df['AGE'] = df['AGE'].apply(convert_age)

### Define a function to apply custom rules for missing values

In [45]:
def fill_intensity_missing(row):
    if pd.isna(row['INTENSITY']):
        if 30 <= row['AGE'] <= 40:
            return 'Moderate'
        else:
            return 'Unknown' 
    else:
        return row['INTENSITY']  

df['INTENSITY'] = df.apply(fill_intensity_missing, axis=1)


In [46]:
missing_values = df.isnull().sum()
print("Check Hours coulmn:")
print(missing_values)

Check Hours coulmn:
AGE                     0
BIRTH_DATE              0
HOURS                   0
ORGAN_CLASSIFICATION    0
REPORT_DATE             0
MCROA_SENSITIVE         0
ANTIBIOTIC              0
INTENSITY               0
dtype: int64


### Convert a column 'AGE' from Float to integer data type.

In [47]:
df['AGE'].dtype


dtype('float64')

In [48]:
df['AGE'] = df['AGE'].astype(float).fillna(0).astype(int) 
df['AGE'].dtype


dtype('int32')

### Apply a function to transform the values of a 'AGE' and add extra coulmn.


In [49]:
def square_age(age):
    return age ** 2 

df['AGE_squared'] = df['AGE'].apply(square_age)
print("Look 98 square 9604")
df[['AGE', 'AGE_squared']]


Look 98 square 9604


,AGE,AGE_squared
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
28273,98,9604
28274,98,9604
28275,98,9604
28276,98,9604


### Normalize a 'HOURS' using Min-Max scaling.


In [50]:
def min_max_scaling(column):
    return (column - column.min()) / (column.max() - column.min())
df['HOURS_normalized'] = min_max_scaling(df['HOURS'])
df[['HOURS', 'HOURS_normalized']]

,HOURS,HOURS_normalized
0,120.0,0.692308
1,48.0,0.230769
2,48.0,0.230769
3,24.0,0.076923
4,24.0,0.076923
...,...,...
28273,72.0,0.384615
28274,72.0,0.384615
28275,72.0,0.384615
28276,72.0,0.384615


### Standardize a 'HOURS' (z-score normalization).

In [51]:
mean_hours = df['HOURS'].mean()
std_hours = df['HOURS'].std()
df['Standardized Hours'] = (df['HOURS'] - mean_hours) / std_hours
df[['HOURS', 'Standardized Hours']].head()


,HOURS,Standardized Hours
0,120.0,1.262227
1,48.0,-0.474155
2,48.0,-0.474155
3,24.0,-1.052949
4,24.0,-1.052949


### Identify duplicate rows in the DataFrame.


In [52]:
duplicate_rows = df[df.duplicated(keep=False)]
print("Duplicate Rows:")
print(duplicate_rows)


Duplicate Rows:
       AGE BIRTH_DATE  HOURS ORGAN_CLASSIFICATION         REPORT_DATE  \
194     10 2013-03-15  168.0        GRAM -VE RODS 2022-03-25 11:23:49   
206     10 2013-06-17   24.0        GRAM -VE RODS 2022-07-21 11:12:05   
221     10 2013-07-16   48.0        GRAM -VE RODS 2023-07-19 11:46:22   
251     10 2013-08-08  120.0        GRAM -VE RODS 2023-08-15 11:48:18   
256     10 2013-03-15  168.0        GRAM -VE RODS 2022-03-25 11:23:49   
...    ...        ...    ...                  ...                 ...   
28229    9 2014-03-28  144.0        GRAM -VE RODS 2022-04-06 11:51:38   
28232    9 2014-04-06  144.0        GRAM -VE RODS 2022-04-13 10:21:35   
28233    9 2014-04-18   72.0        GRAM -VE RODS 2022-04-22 11:02:47   
28234    9 2014-05-24   24.0        GRAM -VE RODS 2022-05-27 10:14:37   
28259    9 2014-04-04   24.0        GRAM -VE RODS 2023-04-07 10:53:58   

      MCROA_SENSITIVE       ANTIBIOTIC INTENSITY  AGE_squared  \
194    Resistant To :       Ampicillin   U

### Drop duplicate rows based on specific columns.


In [53]:
subset_cols = ['AGE', 'BIRTH_DATE', 'HOURS']
df_subset = df.drop_duplicates(subset=subset_cols)

###  Drop duplicate rows based on all columns

In [54]:
df = df.drop_duplicates()
duplicates = df.duplicated()
duplicate_rows = df[duplicates]
print(duplicate_rows)

Empty DataFrame
Columns: [AGE, BIRTH_DATE, HOURS, ORGAN_CLASSIFICATION, REPORT_DATE, MCROA_SENSITIVE, ANTIBIOTIC, INTENSITY, AGE_squared, HOURS_normalized, Standardized Hours]
Index: []


### Convert all string values in a column to lowercase.


In [55]:
df['MCROA_SENSITIVE'] = df['MCROA_SENSITIVE'].str.lower()
print(df['MCROA_SENSITIVE'])

0        resistant to :
1        resistant to :
2        resistant to :
3        resistant to :
4        sensitive to :
              ...      
28273    resistant to :
28274    sensitive to :
28275    resistant to :
28276    resistant to :
28277    sensitive to :
Name: MCROA_SENSITIVE, Length: 26186, dtype: object


### Remove leading and trailing spaces from string values in a column.


In [56]:
df['MCROA_SENSITIVE'] = df['MCROA_SENSITIVE'].str.strip()
print(df['MCROA_SENSITIVE'])

0        resistant to :
1        resistant to :
2        resistant to :
3        resistant to :
4        sensitive to :
              ...      
28273    resistant to :
28274    sensitive to :
28275    resistant to :
28276    resistant to :
28277    sensitive to :
Name: MCROA_SENSITIVE, Length: 26186, dtype: object


###  Replace a specific substring in the column

In [57]:
df['ORGAN_CLASSIFICATION']

0        GRAM -VE RODS
1        GRAM -VE RODS
2        GRAM -VE RODS
3        GRAM -VE RODS
4        GRAM -VE RODS
             ...      
28273    GRAM -VE RODS
28274    GRAM -VE RODS
28275    GRAM -VE RODS
28276    GRAM -VE RODS
28277    GRAM -VE RODS
Name: ORGAN_CLASSIFICATION, Length: 26186, dtype: object

In [58]:
df['ORGAN_CLASSIFICATION'] = df['ORGAN_CLASSIFICATION'].str.replace('VE', 'VH')
print(df['ORGAN_CLASSIFICATION'])

0        GRAM -VH RODS
1        GRAM -VH RODS
2        GRAM -VH RODS
3        GRAM -VH RODS
4        GRAM -VH RODS
             ...      
28273    GRAM -VH RODS
28274    GRAM -VH RODS
28275    GRAM -VH RODS
28276    GRAM -VH RODS
28277    GRAM -VH RODS
Name: ORGAN_CLASSIFICATION, Length: 26186, dtype: object


### Extract a substring 'RODS' from ORGAN_CLASSIFICATION


In [59]:
df['RODS'] = df['ORGAN_CLASSIFICATION'].str.extract('(RODS)', expand=False)
print(df[['ORGAN_CLASSIFICATION', 'RODS']])

      ORGAN_CLASSIFICATION  RODS
0            GRAM -VH RODS  RODS
1            GRAM -VH RODS  RODS
2            GRAM -VH RODS  RODS
3            GRAM -VH RODS  RODS
4            GRAM -VH RODS  RODS
...                    ...   ...
28273        GRAM -VH RODS  RODS
28274        GRAM -VH RODS  RODS
28275        GRAM -VH RODS  RODS
28276        GRAM -VH RODS  RODS
28277        GRAM -VH RODS  RODS

[26186 rows x 2 columns]


### Convert a ' BIRTH_DATE' to datetime format.


In [60]:
df['BIRTH_DATE'] = pd.to_datetime(df['BIRTH_DATE'], errors='coerce')


### Extract year, month, and day from 'BIRTH_DATE' into new columns

In [61]:
df['YEAR'] = df['BIRTH_DATE'].dt.year
df['MONTH'] = df['BIRTH_DATE'].dt.month
df['DAY'] = df['BIRTH_DATE'].dt.day
print(df[['BIRTH_DATE', 'YEAR', 'MONTH', 'DAY']])


      BIRTH_DATE  YEAR  MONTH  DAY
0     2022-10-03  2022     10    3
1     2022-02-03  2022      2    3
2     2022-11-08  2022     11    8
3     2014-06-06  2014      6    6
4     2021-12-22  2021     12   22
...          ...   ...    ...  ...
28273 1925-07-31  1925      7   31
28274 1925-07-31  1925      7   31
28275 1925-07-31  1925      7   31
28276 1925-07-31  1925      7   31
28277 1925-07-31  1925      7   31

[26186 rows x 4 columns]


### Filter rows in 'BIRTH_DATE' based on a date range.


In [62]:
start_date = '2022-01-01'
end_date = '2022-3-31'
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
filtered_df = df[(df['BIRTH_DATE'] >= start_date) & (df['BIRTH_DATE'] <= end_date)]
columns_to_display = [ 'BIRTH_DATE']
filtered_columns_df = filtered_df[columns_to_display]
print(filtered_columns_df)

    BIRTH_DATE
1   2022-02-03
15  2022-02-03
17  2022-03-10
22  2022-02-03
34  2022-03-10
97  2022-02-07
122 2022-02-07
130 2022-02-07
131 2022-03-10
135 2022-02-07
148 2022-02-07
149 2022-02-03
150 2022-02-07
151 2022-03-10
159 2022-02-03
162 2022-03-10
183 2022-02-03


### Convert a categorical column to numerical using one-hot encoding

In [63]:
unique_intensity_values = df['INTENSITY'].unique()
print(unique_intensity_values)

['Heavy' 'Light' 'Unknown' 'Moderate' 'Heavy Mixed' 'Moderate Mixed'
 'Light Mixed']


In [64]:
df = df[df['INTENSITY'] != 'Unknown']
unique_intensity_values = df['INTENSITY'].unique()
print(unique_intensity_values)

['Heavy' 'Light' 'Moderate' 'Heavy Mixed' 'Moderate Mixed' 'Light Mixed']


In [65]:
df_encoded = pd.get_dummies(df, columns=['INTENSITY'])

intensity_columns = [col for col in df_encoded.columns if 'INTENSITY' in col]
df_encoded[intensity_columns]

,INTENSITY_Heavy,INTENSITY_Heavy Mixed,INTENSITY_Light,INTENSITY_Light Mixed,INTENSITY_Moderate,INTENSITY_Moderate Mixed
0,True,False,False,False,False,False
1,False,False,True,False,False,False
2,True,False,False,False,False,False
3,True,False,False,False,False,False
4,True,False,False,False,False,False
...,...,...,...,...,...,...
28273,True,False,False,False,False,False
28274,True,False,False,False,False,False
28275,True,False,False,False,False,False
28276,True,False,False,False,False,False


### Convert a categorical column to numerical using label encoding.

In [66]:
unique_intensity_values = df['ORGAN_CLASSIFICATION'].unique()
print(unique_intensity_values)

['GRAM -VH RODS' 'GRAM NEGATIVH BACILLI' 'GRAM -VH COCCI'
 'Gram Positive Cocci']


In [67]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['ORGAN_CLASSIFICATION_Encoded'] = label_encoder.fit_transform(df['ORGAN_CLASSIFICATION'])
print(df[['ORGAN_CLASSIFICATION', 'ORGAN_CLASSIFICATION_Encoded']])

      ORGAN_CLASSIFICATION  ORGAN_CLASSIFICATION_Encoded
0            GRAM -VH RODS                             1
1            GRAM -VH RODS                             1
2            GRAM -VH RODS                             1
3            GRAM -VH RODS                             1
4            GRAM -VH RODS                             1
...                    ...                           ...
28273        GRAM -VH RODS                             1
28274        GRAM -VH RODS                             1
28275        GRAM -VH RODS                             1
28276        GRAM -VH RODS                             1
28277        GRAM -VH RODS                             1

[23204 rows x 2 columns]


In [68]:
unique_intensity_values = df['ORGAN_CLASSIFICATION_Encoded'].unique()
print(unique_intensity_values)

[1 2 0 3]


### Group values in a categorical column and create a new column with grouped categories.


In [69]:
def group_antibiotic(antibiotic):
    if 'Cef' in antibiotic:
        return 'Cephalosporins'
    elif 'Amox' in antibiotic or 'Amp' in antibiotic:
        return 'Penicillins'
    elif 'Imi' in antibiotic or 'Mero' in antibiotic:
        return 'Carbapenems'
    elif 'Floxacin' in antibiotic:
        return 'Fluoroquinolones'
    elif 'Mycin' in antibiotic or 'Micin' in antibiotic:
        return 'Aminoglycosides'
    else:
        return 'Others'

df['ANTIBIOTIC_Grouped'] = df['ANTIBIOTIC'].apply(group_antibiotic)
print(df[['ANTIBIOTIC_Grouped']])


      ANTIBIOTIC_Grouped
0            Penicillins
1            Penicillins
2            Penicillins
3                 Others
4                 Others
...                  ...
28273             Others
28274        Carbapenems
28275             Others
28276     Cephalosporins
28277        Carbapenems

[23204 rows x 1 columns]


### Create a new column based on existing columns.

In [70]:
def categorize_age(age):
    if age < 13:
        return 'child'
    elif 13 <= age < 20:
        return 'teen'
    elif 20 <= age < 65:
        return 'adult'
    else:
        return 'senior'


df['AGE_Group'] = df['AGE'].apply(categorize_age)
print(df[['AGE', 'AGE_Group']])


       AGE AGE_Group
0        1     child
1        1     child
2        1     child
3        1     child
4        1     child
...    ...       ...
28273   98    senior
28274   98    senior
28275   98    senior
28276   98    senior
28277   98    senior

[23204 rows x 2 columns]


### Discretize a continuous column into bins.

In [71]:
bins = [0, 50, 100, 150, 200]
labels = ['0-50', '51-100', '101-150', '151-200']
df['HOURS_bin'] = pd.cut(df['HOURS'], bins=bins, labels=labels)
print(df[['HOURS', 'HOURS_bin']])

       HOURS HOURS_bin
0      120.0   101-150
1       48.0      0-50
2       48.0      0-50
3       24.0      0-50
4       24.0      0-50
...      ...       ...
28273   72.0    51-100
28274   72.0    51-100
28275   72.0    51-100
28276   72.0    51-100
28277   72.0    51-100

[23204 rows x 2 columns]


### Create polynomial features from existing numerical columns.


### Method1

In [74]:
from sklearn.preprocessing import PolynomialFeatures
degree = 2 
poly = PolynomialFeatures(degree=degree, include_bias=False)
poly_features = poly.fit_transform(df[['AGE', 'HOURS']])

num_features = poly_features.shape[1]
poly_columns = [f'poly_{i}' for i in range(num_features)]
df_poly = pd.DataFrame(poly_features, columns=poly_columns)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df, df_poly], axis=1)
df

,AGE,HOURS,poly_0,poly_1,poly_2,poly_3,poly_4,poly_0,poly_1,poly_2,poly_3,poly_4
0,25,10,25.0,10.0,625.0,250.0,100.0,25.0,10.0,625.0,250.0,100.0
1,30,15,30.0,15.0,900.0,450.0,225.0,30.0,15.0,900.0,450.0,225.0
2,30,20,30.0,20.0,900.0,600.0,400.0,30.0,20.0,900.0,600.0,400.0
3,35,15,35.0,15.0,1225.0,525.0,225.0,35.0,15.0,1225.0,525.0,225.0


### Method2

In [75]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

data = {
    'AGE': [25, 30, 30, 35],
    'HOURS': [10, 15, 20, 15]
}

df = pd.DataFrame(data)


degree = 2
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('poly_features', PolynomialFeatures(degree=degree, include_bias=False))
])

poly_features = pipeline.fit_transform(df[['AGE', 'HOURS']])
num_features = poly_features.shape[1]
poly_columns = [f'poly_{i}' for i in range(num_features)]
df_poly = pd.DataFrame(poly_features, columns=poly_columns)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df, df_poly], axis=1)
df



,AGE,HOURS,poly_0,poly_1,poly_2,poly_3,poly_4
0,25,10,25.0,10.0,625.0,250.0,100.0
1,30,15,30.0,15.0,900.0,450.0,225.0
2,30,20,30.0,20.0,900.0,600.0,400.0
3,35,15,35.0,15.0,1225.0,525.0,225.0


### Example with Built-in Datasets (Iris and Titanic)

In [76]:
import seaborn as sns
import pandas as pd

iris_df = sns.load_dataset('iris')
print("Iris Dataset:")
print(iris_df.head())

titanic_df = sns.load_dataset('titanic')
print("\nTitanic Dataset:")
print(titanic_df.head())


Iris Dataset:
   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa

Titanic Dataset:
   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       Fals

### Merge two DataFrames based on a common column

In [77]:
# Merge based on a numeric index
merged_df = pd.merge(iris_df, titanic_df, left_index=True, right_index=True)

print("\nMerged DataFrame:")
print(merged_df.head())


Merged DataFrame:
   sepal_length  sepal_width  petal_length  petal_width species  survived  \
0           5.1          3.5           1.4          0.2  setosa         0   
1           4.9          3.0           1.4          0.2  setosa         1   
2           4.7          3.2           1.3          0.2  setosa         1   
3           4.6          3.1           1.5          0.2  setosa         1   
4           5.0          3.6           1.4          0.2  setosa         0   

   pclass     sex   age  sibsp  parch     fare embarked  class    who  \
0       3    male  22.0      1      0   7.2500        S  Third    man   
1       1  female  38.0      1      0  71.2833        C  First  woman   
2       3  female  26.0      0      0   7.9250        S  Third  woman   
3       1  female  35.0      1      0  53.1000        S  First  woman   
4       3    male  35.0      0      0   8.0500        S  Third    man   

   adult_male deck  embark_town alive  alone  
0        True  NaN  Southampton 

###  Concatenate vertically

In [78]:
concatenated_vertical = pd.concat([iris_df, titanic_df], axis=0, ignore_index=True)

# Display the concatenated DataFrame vertically
print("\nConcatenated Vertically DataFrame:")
print(concatenated_vertical.head())


Concatenated Vertically DataFrame:
   sepal_length  sepal_width  petal_length  petal_width species  survived  \
0           5.1          3.5           1.4          0.2  setosa       NaN   
1           4.9          3.0           1.4          0.2  setosa       NaN   
2           4.7          3.2           1.3          0.2  setosa       NaN   
3           4.6          3.1           1.5          0.2  setosa       NaN   
4           5.0          3.6           1.4          0.2  setosa       NaN   

   pclass  sex  age  sibsp  parch  fare embarked class  who adult_male deck  \
0     NaN  NaN  NaN    NaN    NaN   NaN      NaN   NaN  NaN        NaN  NaN   
1     NaN  NaN  NaN    NaN    NaN   NaN      NaN   NaN  NaN        NaN  NaN   
2     NaN  NaN  NaN    NaN    NaN   NaN      NaN   NaN  NaN        NaN  NaN   
3     NaN  NaN  NaN    NaN    NaN   NaN      NaN   NaN  NaN        NaN  NaN   
4     NaN  NaN  NaN    NaN    NaN   NaN      NaN   NaN  NaN        NaN  NaN   

  embark_town alive alone 

###  Concatenate horizontally

In [79]:
concatenated_horizontal = pd.concat([iris_df, titanic_df], axis=1)

# Display the concatenated DataFrame horizontally
print("\nConcatenated Horizontally DataFrame:")
print(concatenated_horizontal.head())



Concatenated Horizontally DataFrame:
   sepal_length  sepal_width  petal_length  petal_width species  survived  \
0           5.1          3.5           1.4          0.2  setosa         0   
1           4.9          3.0           1.4          0.2  setosa         1   
2           4.7          3.2           1.3          0.2  setosa         1   
3           4.6          3.1           1.5          0.2  setosa         1   
4           5.0          3.6           1.4          0.2  setosa         0   

   pclass     sex   age  sibsp  parch     fare embarked  class    who  \
0       3    male  22.0      1      0   7.2500        S  Third    man   
1       1  female  38.0      1      0  71.2833        C  First  woman   
2       3  female  26.0      0      0   7.9250        S  Third  woman   
3       1  female  35.0      1      0  53.1000        S  First  woman   
4       3    male  35.0      0      0   8.0500        S  Third    man   

   adult_male deck  embark_town alive  alone  
0        True

# Additional Questions

###  Correaltion Matric

In [80]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
print(df.head())


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


In [81]:
correlation_matrix = df.corr()
print("Correlation Matrix:")
print(correlation_matrix)


Correlation Matrix:
                   sepal length (cm)  sepal width (cm)  petal length (cm)  \
sepal length (cm)           1.000000         -0.117570           0.871754   
sepal width (cm)           -0.117570          1.000000          -0.428440   
petal length (cm)           0.871754         -0.428440           1.000000   
petal width (cm)            0.817941         -0.366126           0.962865   

                   petal width (cm)  
sepal length (cm)          0.817941  
sepal width (cm)          -0.366126  
petal length (cm)          0.962865  
petal width (cm)           1.000000  


### Identification of OUTLIERS and remove it

In [87]:
import seaborn as sns
tips = sns.load_dataset('tips')
print(tips.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [88]:
import numpy as np
z_scores = np.abs((tips[['total_bill', 'tip', 'size']] - tips[['total_bill', 'tip', 'size']].mean()) / tips[['total_bill', 'tip', 'size']].std())
threshold = 3
outliers = tips[(z_scores > threshold).any(axis=1)]
print("\nOutliers identified:")
print(outliers)



Outliers identified:
     total_bill    tip     sex smoker   day    time  size
23        39.42   7.58    Male     No   Sat  Dinner     4
59        48.27   6.73    Male     No   Sat  Dinner     4
125       29.80   4.20  Female     No  Thur   Lunch     6
141       34.30   6.70    Male     No  Thur   Lunch     6
143       27.05   5.00  Female     No  Thur   Lunch     6
156       48.17   5.00    Male     No   Sun  Dinner     6
170       50.81  10.00    Male    Yes   Sat  Dinner     3
212       48.33   9.00    Male     No   Sat  Dinner     4


In [89]:
# Remove outliers
tips_no_outliers = tips[(z_scores <= threshold).all(axis=1)]
print("\nDataFrame without outliers:")
print(tips_no_outliers)



DataFrame without outliers:
     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[236 rows x 7 columns]


###  demonstrate different ways to explore relationships between categorical variables in the Titanic dataset

In [98]:
import pandas as pd
from scipy.stats import chi2_contingency, fisher_exact
import seaborn as sns


titanic = sns.load_dataset('titanic')

print("Original DataFrame:")
print(titanic.head())


print("\nCross-tabulation between 'sex' and 'survived':")
cross_tab_sex_survived = pd.crosstab(titanic['sex'], titanic['survived'])
print(cross_tab_sex_survived)


print("\nChi-square test between 'sex' and 'survived':")
chi2_sex_survived, p_sex_survived, dof_sex_survived, expected_sex_survived = chi2_contingency(cross_tab_sex_survived)
print(f"Chi-square value: {chi2_sex_survived}")
print(f"P-value: {p_sex_survived}")


print("\nFisher's exact test between 'sex' and 'survived':")
odds_ratio_sex_survived, p_fisher_sex_survived = fisher_exact(cross_tab_sex_survived)
print(f"Odds ratio: {odds_ratio_sex_survived}")
print(f"P-value: {p_fisher_sex_survived}")


print("\nCross-tabulation between 'pclass' and 'survived':")
cross_tab_pclass_survived = pd.crosstab(titanic['pclass'], titanic['survived'])
print(cross_tab_pclass_survived)


print("\nChi-square test between 'pclass' and 'survived':")
chi2_pclass_survived, p_pclass_survived, dof_pclass_survived, expected_pclass_survived = chi2_contingency(cross_tab_pclass_survived)
print(f"Chi-square value: {chi2_pclass_survived}")
print(f"P-value: {p_pclass_survived}")



Original DataFrame:
   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  

Cross-tabulation between 'sex' and 'survived':
survived    0    1
sex               
female     81  233
male      468  109

Chi-square test between 'sex' and 'survived':
Chi-square value: 260.717020